In [17]:
import time
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import image_uris
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

# =========================================
# KONFIGURASI
# =========================================
region = "us-east-1"
role = "arn:aws:iam::564415061686:role/service-role/AmazonSageMakerAdminIAMExecutionRole"

model_s3_path = (
    "s3://sagemaker-us-east-1-564415061686/"
    "pytorch-training-2025-12-30-18-09-16-410/output/model.tar.gz"
)

# =========================================
# SESSION
# =========================================
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

# =========================================
# GPU IMAGE
# =========================================
image_uri = image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="2.0",
    py_version="py310",
    image_scope="inference",
    instance_type="ml.g4dn.xlarge",
)

# =========================================
# MODEL (ANTI TIMEOUT CONFIG)
# =========================================
pytorch_model = PyTorchModel(
    model_data=model_s3_path,
    role=role,
    image_uri=image_uri,
    entry_point="sagemaker_inference.py",
    source_dir="../src",
    sagemaker_session=sagemaker_session,
    env={
        "TS_DEFAULT_RESPONSE_TIMEOUT": "600",
        "TS_MAX_RESPONSE_SIZE": "10000000"
    }
)


endpoint_name = "kulitai-gpu-prod"  

print("🚀 Deploying GPU endpoint:", endpoint_name)

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name,
    serializer=IdentitySerializer(content_type="application/octet-stream"),
    deserializer=JSONDeserializer(),
    container_startup_health_check_timeout=600,
    wait=True
)

print("✅ Endpoint siap:", predictor.endpoint_name)


🚀 Deploying GPU endpoint: kulitai-gpu-prod
----------!✅ Endpoint siap: kulitai-gpu-prod
